In [540]:
import numpy as np
import pandas as pd
import ast

In [541]:
movies_df = pd.read_csv("movies.csv")
credits_df = pd.read_csv("credits.csv")

In [542]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [543]:
movies_df = movies_df.merge(credits_df, on='title')

In [544]:
movies_df.shape

(4808, 23)

In [545]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4808 entries, 0 to 4807
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4808 non-null   int64  
 1   genres                4808 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4808 non-null   int64  
 4   keywords              4808 non-null   object 
 5   original_language     4808 non-null   object 
 6   original_title        4808 non-null   object 
 7   overview              4805 non-null   object 
 8   popularity            4808 non-null   float64
 9   production_companies  4808 non-null   object 
 10  production_countries  4808 non-null   object 
 11  release_date          4807 non-null   object 
 12  revenue               4808 non-null   int64  
 13  runtime               4806 non-null   float64
 14  spoken_languages      4808 non-null   object 
 15  status               

In [546]:
movies_df = movies_df[['movie_id','title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [547]:
movies_df.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [548]:
movies_df.dropna(inplace=True)

In [549]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [550]:
import ast

In [551]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
      L.append(i['name'])
    return L

In [552]:
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)

In [553]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [554]:
movies_df['cast'] = movies_df['cast'].apply(convert3)

In [555]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [556]:
movies_df['crew'] = movies_df['crew'].apply(fetch_director)

KeyboardInterrupt: 

In [520]:
movies_df['overview'] = movies_df['overview'].apply(lambda x: x.split())

In [521]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

In [522]:
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

In [523]:
new_df = movies_df[['movie_id', 'title', 'tags']]


In [524]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\pande\AppData\Local\Temp\ipykernel_12204\1824047427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [525]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

C:\Users\pande\AppData\Local\Temp\ipykernel_12204\1380776331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [526]:
from sklearn.feature_extraction.text import CountVectorizer as cv

cv = cv(max_features=5000, stop_words='english')

In [527]:
cv.fit_transform(new_df['tags']).toarray().shape

(4805, 5000)

In [528]:
vector = cv.fit_transform(new_df['tags']).toarray()
vector[0]

array([0, 0, 0, ..., 0, 0, 0])

In [529]:
len(cv.get_feature_names_out())

5000

In [530]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [531]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
  
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\pande\AppData\Local\Temp\ipykernel_12204\2192781525.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [532]:
from sklearn.metrics.pairwise import cosine_similarity

In [533]:
cosine_similarity(vector)

array([[1.        , 0.08980265, 0.05986843, ..., 0.02457366, 0.02635231,
        0.        ],
       [0.08980265, 1.        , 0.06451613, ..., 0.02648136, 0.        ,
        0.        ],
       [0.05986843, 0.06451613, 1.        , ..., 0.02648136, 0.        ,
        0.        ],
       ...,
       [0.02457366, 0.02648136, 0.02648136, ..., 1.        , 0.06993786,
        0.04783649],
       [0.02635231, 0.        , 0.        , ..., 0.06993786, 1.        ,
        0.05129892],
       [0.        , 0.        , 0.        , ..., 0.04783649, 0.05129892,
        1.        ]])

In [534]:
cosine_similarity(vector).shape

(4805, 4805)

In [535]:
similarity = cosine_similarity(vector)

In [536]:
similarity[0].shape

(4805,)

In [537]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6]

[(539, np.float64(0.25724787771376323)),
 (1192, np.float64(0.2545875386086578)),
 (260, np.float64(0.24759378423606915)),
 (507, np.float64(0.24720661623652207)),
 (1214, np.float64(0.24595492912420727))]

In [538]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [559]:
recommend('Iron Man')

Iron Man 2
Iron Man 3
Avengers: Age of Ultron
Captain America: Civil War
The Avengers
